# Gotchas of BrainPy Transformations

In [1]:
import brainpy as bp
import brainpy.math as bm

bm.set_platform('cpu')

bp.__version__

'2.4.2'

BrainPy provides a novel concept for object-oriented transformations based [brainpy.math.Variable](../tutorial_math/variables.ipynb). However, this kind of transformations faces several gotchas:

## 1. ``Variable`` that will be changed cannot be functional arguments

This will not work too for the new oo transformations.

In [2]:
@bm.jit
def f(a, b):
  a.value = b

a = bm.Variable(bm.ones(1))
b = bm.Variable(bm.ones(1) * 10)

In [3]:
f(a, b)

try:
  assert bm.allclose(a, b)
  print('a equals to b.')
except:
  print('a is not equal to b.')

a is not equal to b.


In [4]:
a

Variable(value=Array([1.]), dtype=float32)

All ``Variable``s should be used in a global context.

Instead, this works:

In [5]:
@bm.jit
def f(b):
  a.value = b

a = bm.Variable(bm.ones(1))
b = bm.Variable(bm.ones(1) * 10)



In [6]:
f(b)

a

Variable(value=Array([10.]), dtype=float32)

## 2. Functions to be transformed are called twice

The core mechanism of any brainpy transformation is that it firsts calls the function to automatically find all ``Variable``s used in the model, and then it calls the function again to compile the model with the found ``Variable``s.

Therefore, any function that the user create will be called more than twice.

In [7]:
@bm.jit
def f(inp):
  print('calling f ...')
  return inp

@bm.jit
def g(inp):
  print('calling g ...')
  return f(inp)

Taking the above function as an example, when we use this function, we will get:

In [8]:
g(1.)

calling g ...
calling f ...
calling g ...
calling f ...


Array(1., dtype=float32, weak_type=True)

It sequentially calls ``f`` and ``g`` to infer all dynamical variables (instances of ``Variable``) used in these two functions. So we got first two lines of ``calling g ...`` and ``calling f``.

Then, it compiles the two functions, so that we got next two lines of ``calling g ...`` and ``calling f``.

Note that this property may get what are not correct in the Python level variables. For example, when we use a global variable to record the number of times the function called:

In [9]:
num = [0]

@bm.jit
def h(inp):
  num[0] += 1
  return inp

In [10]:
h(1.)

Array(1., dtype=float32, weak_type=True)

Although we called the function ``h`` once, we got the number of ``2``.

In [11]:
num

[2]